In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install -q transformers

In [ ]:
pip install -q shap

In [ ]:
import pandas as pd
import numpy as np
import torch
import shap
import pickle

In [ ]:
device = torch.device("cuda")

In [ ]:
test =  pd.read_csv('/content/drive/MyDrive/MasterThesis/Dataset/X_test_all.csv')
test_sentences = test['clean_review']

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
model = torch.load('/content/drive/MyDrive/MasterThesis/BERT/bert_1', map_location="cuda:0")

In [ ]:
import scipy as sp
# define a prediction function
def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=100, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val

In [ ]:
# build an explainer using a token masker
explainer = shap.Explainer(f, tokenizer)

In [ ]:
#{845, 1556, 1750, 2150, 2514, 2631, 2652, 2845, 3354, 3447, 3929} 

#{1276, 405, 1374, 2047} ---neg
#{3145, 1291, 2124, 3031, 3356} ---pos

j = 2047 
shap_values = explainer(test['clean_review'][j:j+1], fixed_context=1) #2257

In [ ]:
#we need this information to use later for the survey
#import pickle
#with open('/content/drive/MyDrive/MasterThesis/Dataset/BERT_all_shap_values', 'wb') as f:
 #   pickle.dump(shap_values, f)    

In [ ]:
i=0
shap.plots.text(shap_values[i])

**Sample calculation**

Calculating and saving feature importance score for the reviews used as a demonstration example to measure the proposed criteria.

In [ ]:
shap_v = pickle.load(open('/content/drive/MyDrive/MasterThesis/Dataset/BERT_all_shap_values', 'rb'))   

In [ ]:
pos_i = pickle.load(open('/content/drive/MyDrive/MasterThesis/Dataset/positive_sample_50', 'rb'))
neg_i = pickle.load(open('/content/drive/MyDrive/MasterThesis/Dataset/negative_sample_50', 'rb')) 

In [ ]:
#function to remove None values

def func_remove_none(b):
  l = list(filter(lambda x: x[0] != None, b))
  l = list(filter(lambda x: x[0] != '', l))
  return l

In [ ]:
#shap saves the word list with a space infront, that is why we create
#a function to remove it
def remove_space(b):
  k = []
  for i in b:
    j = i.replace(' ','')
    k.append(j)
  return k

In [ ]:
#map each word with its shap value
def merge_el (el):
  w = []
  for indx in el:
    a = shap_v.values[indx]
    b = shap_v.data[indx]
    b = remove_space(b)
    temp_list = []
    for i in range(0, len(a)):
      temp_list.append((b[i],a[i]))
    w.append(temp_list)
  return w

In [ ]:
#save the data
r1 = merge_el (pos_i)
r2 = merge_el (neg_i)

In [ ]:
def func_sum (sample):
  my_sum = []
  for i in range(len(sample)):
    x = sample[i]
    d = {}
    [d.__setitem__(first, d.get(first, 0) + second) for first, second in x]
    r = list(d.items())
    my_sum.append(r)
  return my_sum

In [ ]:
resp = func_sum(r1)
resn = func_sum(r2)

In [ ]:
r1 = []
for el in resp:
  a = func_remove_none(el)
  a_sorted = sorted(a, key=lambda x: x[1])
  r1.append(a_sorted)

r2 = []
for el in resn:
  a = func_remove_none(el)
  a_sorted = sorted(a, key=lambda x: x[1])
  r2.append(a_sorted)

In [ ]:
with open('/content/drive/MyDrive/MasterThesis/Dataset/bert_shap_positive_sample_50', 'wb') as f:
    pickle.dump(r1, f)

with open('/content/drive/MyDrive/MasterThesis/Dataset/bert_shap_negative_sample_50', 'wb') as f:
    pickle.dump(r2, f)            